In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import operator

In [71]:
columns = ['Artist', 'Title', 'Album', '#_of_views', 'Release_date', '#_feat_artists', 'Artist_pop', 'Genre', 
           'Followers', 'Danceability', 'Energy', 'Valence', 'Duration', 'Loudness', '#_words', 'Lyrics']
music_df = pd.read_csv('lyricDataset.scv', names = columns)

In [72]:
music_df.loc[music_df['Artist'] == 'Travis Scott']

,Artist,Title,Album,#_of_views,Release_date,#_feat_artists,Artist_pop,Genre,Followers,Danceability,Energy,Valence,Duration,Loudness,#_words,Lyrics
1540,Travis Scott,SICKO MODE,ASTROWORLD,7423340,2018-08-03,1,97,rap,8007871,0.823,0.710,0.4460,312820,-3.757,771,Astro yeah\nSun is down freezin' cold\nThat'...
1541,Travis Scott,​goosebumps,Birds in the Trap Sing McKnight,4067071,2016-09-02,1,97,rap,8007871,0.847,0.719,0.4420,243837,-3.300,548,Yeah\n7:30 in the night yeah\nOoh-oooh ooh\n...
1542,Travis Scott,BUTTERFLY EFFECT,ASTROWORLD,2049513,2017-05-15,0,97,rap,8007871,0.764,0.628,0.1960,190557,-5.851,469,Murda on the beat so it's not nice\nOoh hmm\n...
1543,Travis Scott,Antidote,Rodeo (Deluxe Edition),1957421,2015-06-23,0,97,rap,8007871,0.713,0.526,0.1310,262693,-5.046,551,Don't you open up that window\nDon't you let o...
1544,Travis Scott,​beibs in the trap,Birds in the Trap Sing McKnight,1847257,2016-09-02,1,97,rap,8007871,0.743,0.629,0.4570,213863,-5.994,547,That coca\n\nI just poured an 8 in a liter\nTh...
1545,Travis Scott,HIGHEST IN THE ROOM,Introducing iFruit Radio,1800337,2019-10-04,0,97,rap,8007871,0.601,0.425,0.0581,175721,-8.671,303,I got room in my fumes (Yeah)\nShe fill my min...
1546,Travis Scott,STARGAZING,ASTROWORLD,1403238,2018-08-03,0,97,rap,8007871,0.491,0.778,0.0464,270715,-3.578,487,Rollin' rollin' rollin' got me stargazin' (...
1547,Travis Scott,YOSEMITE,ASTROWORLD,1380205,2018-08-03,2,97,rap,8007871,0.783,0.566,0.1490,150043,-6.704,398,Ice on my neck flawless baguettes\nHop off a ...
1548,Travis Scott,CAN’T SAY,ASTROWORLD,1267405,2018-08-03,1,97,rap,8007871,0.703,0.709,0.7120,198408,-5.360,562,No you can't say if I'm mad or not\nSmokin' h...
1549,Travis Scott,​through the late night,Birds in the Trap Sing McKnight,1134501,2016-09-02,1,97,rap,8007871,0.738,0.603,0.2610,286236,-1.368,672,Hmm hmm meh meh\nMmm meh meh meh\nHmm h...


In [73]:
genres = music_df['Genre']
g_list = list(genres)
genre_list = ['country', 'hip hop', 'pop', 'rock', 'punk', 'metal', 'australian'] #Australian reggae is outlier

In [74]:
correct_guesses = {}
for index, genre in enumerate(g_list):
    if type(genre) == float:
        continue
    else:
        if len(genre) == 1:
            print(genre)
            split_genres = genre
        else:
            split_genres = genre.split(' / ') 
    for g in genre_list:
        correct = 0
        if len(split_genres) == 1 and split_genres[0] == 'rap':
            correct_guesses['hip hop'] = correct
        else:
            for split in split_genres:
                if g in split:
                    correct += 1
            correct_guesses[g] = correct
    correct_genre = max(correct_guesses, key=lambda key: correct_guesses[key])
    g_list[index] = correct_genre
    correct_guesses.clear()

In [75]:
music_df['corrected_genres'] = g_list
music_df

,Artist,Title,Album,#_of_views,Release_date,#_feat_artists,Artist_pop,Genre,Followers,Danceability,Energy,Valence,Duration,Loudness,#_words,Lyrics,corrected_genres
0,Miranda Lambert,Vice,The Weight of These Wings,42261,2016-07-18,0,74,contemporary country / country / country dawn ...,2184569,0.568,0.562,0.247,240280,-7.775,233,Sting of the needle dropping on a vinyl\nNeon ...,country
1,Miranda Lambert,Tin Man,The Weight of These Wings,40702,2016-11-18,0,74,contemporary country / country / country dawn ...,2184569,0.425,0.372,0.292,259280,-10.920,147,Hey there Mr. Tin Man\nYou don't know how luck...,country
2,Miranda Lambert,Little Red Wagon,Platinum,29992,2014-06-03,0,74,contemporary country / country / country dawn ...,2184569,0.584,0.813,0.774,204387,-7.221,255,You only love me for my big sunglasses\nAnd my...,country
3,Miranda Lambert,Mama’s Broken Heart,Four the Record,29986,2013-01-14,0,74,contemporary country / country / country dawn ...,2184569,0.768,0.809,0.748,177773,-4.204,315,I cut my bangs with some rusty kitchen scissor...,country
4,Miranda Lambert,Somethin’ Bad,Platinum,26966,2014-05-19,1,74,contemporary country / country / country dawn ...,2184569,0.777,0.245,0.349,109107,-14.266,307,Stand on the bar stomp your feet start clapp...,country
5,Miranda Lambert,Runnin’ Just in Case,The Weight of These Wings,19365,2016-11-18,0,74,contemporary country / country / country dawn ...,2184569,0.420,0.498,0.107,273480,-9.091,240,There's trouble where I'm going\nBut I'm gonna...,country
6,Miranda Lambert,The House That Built Me,Revolution,13652,2010-03-08,0,74,contemporary country / country / country dawn ...,2184569,0.515,0.300,0.206,236627,-8.005,297,"I know they say ""You can’t go home again""\nI j...",country
7,Miranda Lambert,We Should Be Friends,The Weight of These Wings,13393,2016-12-12,0,74,contemporary country / country / country dawn ...,2184569,0.622,0.701,0.777,170747,-6.212,239,If your mind's as cluttered as your kitchen si...,country
8,Miranda Lambert,Over You,Four the Record,12684,2012-01-09,0,74,contemporary country / country / country dawn ...,2184569,0.566,0.507,0.141,253373,-6.973,156,Weather man said it’s gonna snow\nBy now I sh...,country
9,Miranda Lambert,Highway Vagabond,The Weight of These Wings,11586,2016-11-04,0,74,contemporary country / country / country dawn ...,2184569,0.447,0.823,0.909,233267,-7.174,326,I wanna go somewhere where nobody knows\nI wan...,country


In [80]:
music_df.groupby(['corrected_genres']).count()

AttributeError: 'DataFrameGroupBy' object has no attribute 'value_counts'